In [65]:
import pandas as pd
import numpy as np


# Creating the dataframe used for display purposes 

In [66]:
dfd = pd.read_csv('all_seasons.csv',index_col=0)
dfd.reset_index(inplace=True, drop = True)
dfd['comp'].unique()

array(['Premier League', 'EFL Cup', 'Champions Lg', 'FA Cup', 'La Liga',
       'Supercopa de España', 'Copa del Rey', 'Ligue 1',
       'Trophée des Champions', 'Coupe de France', 'Bundesliga',
       'Super Cup', 'DFL-Supercup', 'DFB-Pokal', 'Community Shield',
       'Primeira Liga', 'Serie A', 'Coppa Italia', 'Supercoppa Italiana',
       'Europa Lg', 'First Division A', 'Eredivisie', 'Super League',
       'Süper Lig', 'Superliga', 'NB I', 'Swiss Super League',
       'Allsvenskan', '1. HNL', 'Premiership', 'First League'],
      dtype=object)

In [67]:
dfd = dfd[dfd['comp']=='Champions Lg']
dfd = dfd[dfd['season']=='2022-2023']
dfd.shape

(240, 236)

In [68]:
cols_todel = ['ga','match report_x','match report_y','date_opp','time_opp','comp_opp','round_opp','day_opp','venue_opp',
              'result_opp','ga_opp','opponent_opp','xg', 'xg_y', 'match report_opp','poss_y', 'captain','referee','notes', 'attendance']

In [69]:
dfd.drop(cols_todel, axis=1, inplace=True)
dfd.loc[dfd["gf"].str.contains(" "),'gf'] = dfd["gf"].str[0] # if the Score column starts with "(", then grab the first from left char.
dfd.loc[dfd["gf_opp"].str.contains(" "),'gf_opp'] = dfd["gf_opp"].str[0]
dfd['opponent'] = dfd['opponent'].apply(lambda x: x[3:].lstrip() if x[0].islower() else x)  # If the format of the name starts with small letters, that is the country code. 
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Atletico Madrid' if x == 'Atlético Madrid' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Bayer Leverkusen' if x == 'Leverkusen' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Eintracht Frankfurt' if x == 'Eint Frankfurt' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Internazionale' if x == 'Inter' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Paris Saint Germain' if x == 'Paris S-G' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Red Bull Salzburg' if x == 'RB Salzburg' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Shakhtar Donetsk' if x == 'Shakhtar' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Tottenham Hotspur' if x == 'Tottenham' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Viktoria Plzen' if x == 'Viktoria Plzeň' else x)

dfd['date'] = pd.to_datetime(dfd['date'],dayfirst=True)
dfd["venue_code"] = dfd["venue"].astype("category").cat.codes
dfd["opp_code"] = dfd["opponent"].astype("category").cat.codes #
dfd["hour"] = dfd["time"].str.replace(":.+", "", regex=True).astype("int")
dfd["day_code"] = dfd["date"].dt.dayofweek


In [70]:
dfd.shape

(240, 220)

In [71]:
missing = dfd.isna().mean(axis =0)
missing

date          0.0
time          0.0
comp          0.0
round         0.0
day           0.0
             ... 
season        0.0
venue_code    0.0
opp_code      0.0
hour          0.0
day_code      0.0
Length: 220, dtype: float64

In [72]:
over_threshold = missing[missing >= 0.2]
over_threshold

avgdist        0.2
avgdist_opp    0.2
dtype: float64

In [73]:
dfd.head()

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,won_opp,won%_opp,xga,formation,team,season,venue_code,opp_code,hour,day_code
3032,2022-09-07,21:00,Champions Lg,Group stage,Wed,Away,W,2,Internazionale,1.0,...,13.0,56.5,0.7,4-2-3-1,Bayern Munich,2022-2023,0,17,21,2
3034,2022-09-13,21:00,Champions Lg,Group stage,Tue,Home,W,2,Barcelona,2.0,...,14.0,66.7,2.0,4-2-3-1,Bayern Munich,2022-2023,1,3,21,1
3037,2022-10-04,18:45,Champions Lg,Group stage,Tue,Home,W,5,Viktoria Plzen,5.0,...,12.0,44.4,0.5,4-2-3-1,Bayern Munich,2022-2023,1,45,18,1
3039,2022-10-12,21:00,Champions Lg,Group stage,Wed,Away,W,4,Viktoria Plzen,4.0,...,19.0,50.0,1.2,4-2-3-1,Bayern Munich,2022-2023,0,45,21,2
3043,2022-10-26,21:00,Champions Lg,Group stage,Wed,Away,W,3,Barcelona,3.0,...,4.0,28.6,0.7,4-2-3-1,Bayern Munich,2022-2023,0,3,21,2


In [74]:
dfd['team'].unique()

array(['Bayern Munich', 'Benfica', 'Napoli', 'Real Madrid', 'Chelsea',
       'Manchester City', 'Liverpool', 'Paris Saint Germain', 'Milan',
       'RB Leipzig', 'Internazionale', 'Porto', 'Dortmund',
       'Tottenham Hotspur', 'Club Brugge', 'Eintracht Frankfurt',
       'Barcelona', 'Sporting CP', 'Marseille', 'Shakhtar Donetsk',
       'Red Bull Salzburg', 'Ajax', 'Atletico Madrid', 'Bayer Leverkusen',
       'Sevilla', 'Dinamo Zagreb', 'Juventus', 'FC Copenhagen',
       'Maccabi Haifa', 'Celtic', 'Viktoria Plzen', 'Rangers'],
      dtype=object)

In [75]:
dfd[dfd['team']=='Maccabi Haifa']

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,won_opp,won%_opp,xga,formation,team,season,venue_code,opp_code,hour,day_code
3979,2022-07-20,20:00,Champions Lg,Second qualifying round,Wed,Home,D,1,Olympiacos,1.0,...,NaN,NaN,NaN,4/03/2003,Maccabi Haifa,2022-2023,1,27,20,2
3980,2022-07-27,22:00,Champions Lg,Second qualifying round,Wed,Away,W,4,Olympiacos,4.0,...,NaN,NaN,NaN,4/03/2003,Maccabi Haifa,2022-2023,0,27,22,2
3981,2022-08-03,20:00,Champions Lg,Third qualifying round,Wed,Home,W,4,Apollon,3.0,...,NaN,NaN,NaN,4/03/2003,Maccabi Haifa,2022-2023,1,1,20,2
3982,2022-08-09,20:00,Champions Lg,Third qualifying round,Tue,Away,L,0,Apollon,0.0,...,NaN,NaN,NaN,4/03/2003,Maccabi Haifa,2022-2023,0,1,20,1
3983,2022-08-17,22:00,Champions Lg,Play-off round,Wed,Home,W,3,Red Star,3.0,...,NaN,NaN,NaN,4/03/2003,Maccabi Haifa,2022-2023,1,36,22,2
3984,2022-08-23,21:00,Champions Lg,Play-off round,Tue,Away,D,2,Red Star,1.0,...,NaN,NaN,NaN,3-4-1-2,Maccabi Haifa,2022-2023,0,36,21,1
3985,2022-09-06,20:00,Champions Lg,Group stage,Tue,Away,L,0,Benfica,0.0,...,9.0,45.0,1.1,5/03/2002,Maccabi Haifa,2022-2023,0,6,20,1
3986,2022-09-14,22:00,Champions Lg,Group stage,Wed,Home,L,1,Paris Saint Germain,1.0,...,4.0,30.8,1.8,5/03/2002,Maccabi Haifa,2022-2023,1,29,22,2
3987,2022-10-05,21:00,Champions Lg,Group stage,Wed,Away,L,1,Juventus,1.0,...,5.0,35.7,2.3,5/03/2002,Maccabi Haifa,2022-2023,0,18,21,2
3988,2022-10-11,19:45,Champions Lg,Group stage,Tue,Home,W,2,Juventus,2.0,...,12.0,40.0,0.5,4/03/2003,Maccabi Haifa,2022-2023,1,18,19,1


In [76]:
dfd[dfd.isnull()] # Many rows still have null values. Delete them. 

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,won_opp,won%_opp,xga,formation,team,season,venue_code,opp_code,hour,day_code
3032,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3034,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3037,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3039,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3043,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4072,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4074,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4077,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
a = dfd[dfd['team']=='Maccabi Haifa']

In [78]:
dfd = dfd.dropna(axis = 0)
dfd.shape

(166, 220)

In [79]:
len(dfd['team'].unique())

32

In [80]:
dfd.shape

(166, 220)

In [81]:
dfd.sort_values(by='date').reset_index(inplace=True)
num_cols = dfd.select_dtypes('number').columns
num_cols.has_duplicates

False

In [82]:
new_cols = [f"{c}_rolling" for c in list(num_cols)] # add_rolling to column names
new_cols

['gls_rolling',
 'sh_rolling',
 'sot_rolling',
 'sot%_rolling',
 'g/sh_rolling',
 'g/sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling',
 'xg_x_rolling',
 'npxg_rolling',
 'npxg/sh_rolling',
 'g-xg_rolling',
 'np:g-xg_rolling',
 'gls_opp_rolling',
 'sh_opp_rolling',
 'sot_opp_rolling',
 'sot%_opp_rolling',
 'g/sh_opp_rolling',
 'g/sot_opp_rolling',
 'dist_opp_rolling',
 'fk_opp_rolling',
 'pk_opp_rolling',
 'pkatt_opp_rolling',
 'xg_opp_rolling',
 'npxg_opp_rolling',
 'npxg/sh_opp_rolling',
 'g-xg_opp_rolling',
 'np:g-xg_opp_rolling',
 'sota_rolling',
 'saves_rolling',
 'save%_rolling',
 'cs_rolling',
 'psxg_rolling',
 'psxg+/-_rolling',
 'pka_rolling',
 'pksv_rolling',
 'pkm_rolling',
 'cmp_rolling',
 'att_rolling',
 'cmp%_rolling',
 'thr_rolling',
 'launch%_rolling',
 'avglen_rolling',
 'opp_rolling',
 'stp_rolling',
 'stp%_rolling',
 '#opa_rolling',
 'avgdist_rolling',
 'sota_opp_rolling',
 'saves_opp_rolling',
 'save%_opp_rolling',
 'cs_opp_rolling',
 'ps

In [83]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3,closed="left").mean() # closed = left -> to avoid data leakage from future matches
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [84]:
dfd_rolling = dfd.groupby('team').apply(lambda x: rolling_averages(x,num_cols,new_cols))
dfd_rolling = dfd_rolling.droplevel('team')

C:\Users\Ramin\AppData\Local\Temp\ipykernel_8540\1506766940.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group[new_cols] = rolling_stats
C:\Users\Ramin\AppData\Local\Temp\ipykernel_8540\1506766940.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group[new_cols] = rolling_stats
C:\Users\Ramin\AppData\Local\Temp\ipykernel_8540\1506766940.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [85]:
len(dfd_rolling['team'].unique())

28

In [86]:
len(dfd_rolling[dfd_rolling['season']=='2022-2023']['team'].unique())

28

In [87]:
dfd_rolling[dfd_rolling['team']=='Maccabi Haifa']

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,pkcon_opp_rolling,og_opp_rolling,recov_opp_rolling,won_opp_rolling,won%_opp_rolling,xga_rolling,venue_code_rolling,opp_code_rolling,hour_rolling,day_code_rolling
3988,2022-10-11,19:45,Champions Lg,Group stage,Tue,Home,W,2,Juventus,2.0,...,0.0,0.0,52.000000,6.0,37.166667,1.733333,0.333333,17.666667,21.000000,1.666667
3989,2022-10-25,21:00,Champions Lg,Group stage,Tue,Away,L,2,Paris Saint Germain,2.0,...,0.0,0.0,53.333333,7.0,35.500000,1.533333,0.666667,21.666667,20.666667,1.666667


## Minmax normaliation for the radar plots. 

In [88]:
dfd_rolling[new_cols] = (dfd_rolling[new_cols]-dfd_rolling[new_cols].min())/(dfd_rolling[new_cols].max()-dfd_rolling[new_cols].min())

In [89]:
dfd_rolling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 3757 to 4048
Columns: 427 entries, date to day_code_rolling
dtypes: datetime64[ns](1), float64(408), int32(1), int64(3), int8(2), object(12)
memory usage: 236.2+ KB


In [92]:
dfd_rolling.to_csv('dfd_rolling.csv')


In [ ]:
len(dfd_rolling['team'].unique())

#### Useful stuff
Do the following to get a list of the numeric columns, or select an type of column based on its tpes. 

```[key for key in dict(DF.dtypes) if dict(DF.dtypes)[key] in ['float64', 'int64']]```

predictors2 = [b for a, b in zip(sfs.get_support(), all_cols) if a] # extracting the features sfs has chosen for us. all_cols is a string list of all the columns, this uses the sfs.get_support mask.